<a href="https://colab.research.google.com/github/nachenz/Ignacio-Gonzalez/blob/main/Proyecto_Final_Integrador_DATA_SCIENCE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Final Integrador DATA SCIENCE



## 1. Cargo los dataframes mediante API de Kaggle (ETL)\:

In [ ]:
from google.colab import files
files.upload()

1.2 Cargo e importo las librerias y modelos que voy a usar a lo largo del proyecto

In [ ]:
import os
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import spacy
import spacy
from wordcloud import WordCloud
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
!pip install --upgrade spacy

!python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

In [ ]:

os.makedirs("/root/.kaggle", exist_ok=True)
!mv kaggle.json /root/.kaggle/

# Dar permisos adecuados
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d zynicide/wine-reviews


In [ ]:
with zipfile.ZipFile("wine-reviews.zip", "r") as zip_ref:
    zip_ref.extractall("wines_data")


1.1 Defino variables para cada uno de los archivos .csv que componen el .zip linkeado anteriormente

In [ ]:
df1 = pd.read_csv("wines_data/winemag-data-130k-v2.csv")
df2 = pd.read_csv("wines_data/winemag-data_first150k.csv")


# 2. Análisis exploratorio de datos (EDA)

In [ ]:
df1.head()

***Veo descripción de variables numericas del df1***

In [ ]:
df1.describe()

In [ ]:
filas, columnas = df1.shape
print(f"El DataFrame tiene {filas} filas y {columnas} columnas.")

## 2.1 Observo nulos

In [ ]:
print(df1.isnull().sum())

***Al haber pocos nulos en la columna "precie", opto por eliminar las filas donde no hayan valores. A su vez, los nulos de "region_1" los reemplazo por "unknown"***

In [ ]:
df1['region_1'] = df1['region_1'].fillna('unknown')
df1_nuevo = df1.dropna(subset=["price"])

***Veo descripción de variables numericas del df2***

In [ ]:
df2.describe()

In [ ]:
filas, columnas = df1.shape
print(f"El DataFrame tiene {filas} filas y {columnas} columnas.")

***Elimino aquellas columnas que no creo que sean relevantes para el analisis.***

In [ ]:
df2 = df2.drop(columns=["region_2", "Unnamed: 0"])
df1 = df1.drop(columns=["region_2", "taster_twitter_handle", "Unnamed: 0"])

***2.2 Observo nulos***

In [ ]:
print(df2.isnull().sum())

***Hago lo mismo que hice con la columna "region_1" en el DataFrame 1***

In [ ]:
df2['region_1'] = df2['region_1'].fillna('Unknow')

***Elimino filas donde "columna1" tiene***


In [ ]:
df2_nuevo = df2.dropna(subset=["price"])

##***Concateno ambos df para que me quede uno solo***

In [ ]:
df_final = pd.concat([df1_nuevo, df2_nuevo], ignore_index=True)

In [ ]:
df_final.head()

In [ ]:
filas, columnas = df_final.shape
print(f"El DataFrame tiene {filas} filas y {columnas} columnas.")

## Observo los nulos totales que quedaron con la concatenación

In [ ]:
print(df_final.isnull().sum())

## Eliminar filas con valores nulos en columnas categóricas


***En este caso, elimino las filas que tienen valores nulos en las columnas 'variety' y 'province' por ser una cantidad pequeña.***

In [ ]:
df_final_limpio = df_final.dropna(subset=['variety', 'province'])


In [ ]:
print(df_final_limpio.isnull().sum())

***Aquellas columnas categoricas que tienen nulos los completo con 'Unknow'***

In [ ]:
df1['designation'] = df1['designation'].fillna('Unknow')
df1['region_1'] = df1['region_1'].fillna('Unknow')
df1['taster_name'] = df1['taster_name'].fillna('Unknow')
df1['title'] = df1['title'].fillna('Unknow')

#3. Graficos de la data

In [ ]:
columna_a_graficar = "points"
columna = df_final_limpio[columna_a_graficar]

plt.figure(figsize=(6, 4))
sns.histplot(data=columna, bins=30, kde=True)
plt.title(f"Histograma de puntaje")
plt.show()


 *⬆️ Podemos observar que la puntuación mas baja es de 80.0 con una alza notoria en los 87.0 puntos*







In [ ]:
conteo_puntos = df_final_limpio["points"]

plt.figure(figsize=(10,6))
conteo_puntos.value_counts().plot(kind='bar', color='skyblue', edgecolor='black')
plt.xlabel("Puntaje")
plt.ylabel("Frecuencia")
plt.title("Frecuencia de puntos asignados a los vinos")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

sns.boxplot(x=conteo_puntos.values)
plt.title(f"Grafico de bigotes para ver puntajes outliers")
plt.show()

 ⬆️ *Observamos cual puntuación es la que más se repite en estos df y que números pueden integrar valores conocidos como ouliers*







In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(data=df_final_limpio, x="taster_name", order=df_final_limpio["taster_name"].value_counts().index)
plt.xticks(rotation=80)  # Rotar etiquetas si hay muchos nombres
plt.title("Frecuencia de cada catador")
plt.xlabel("Catador")
plt.ylabel("Frecuencia")
plt.show()


 *⬆️ El catador que mas opiniones realizó fue Roger Voss*

##A continuación, grafico los 10 valores mas frecuentes en los **paises**, **viaredad de uva** y **viñedo**

In [ ]:
conteo = df_final_limpio["country"].value_counts().head(8)

plt.figure(figsize=(9, 9))
conteo.plot(kind="pie", autopct="%1.1f%%", colors=plt.cm.Paired.colors, startangle=90)
plt.title("Distribución de los 10 paises mas frecuentes")
plt.ylabel("")
plt.show()


In [ ]:
conteo = df_final_limpio["variety"].value_counts().head(10)

plt.figure(figsize=(7, 7))
conteo.plot(kind="pie", autopct="%1.1f%%", colors=plt.cm.Paired.colors, startangle=90)
plt.title("Distribución de las 10 uvas más frecuentes")
plt.ylabel("")
plt.show()


In [ ]:
conteo = df_final_limpio["winery"].value_counts().head(10)

plt.figure(figsize=(7, 7))
conteo.plot(kind="pie", autopct="%1.1f%%", colors=plt.cm.Paired.colors, startangle=90)
plt.title("Distribución de los 10 viñedos más frecuentes")
plt.ylabel("")
plt.show()

# 2. Análisis de Precios y Factores de Calidad


In [ ]:
conteo_precio = df_final_limpio["price"]

sns.boxplot(x=conteo_precio.values)
plt.title(f"Boxplot para variable precios de los vinos")
plt.show()


⬆️ ***Genero un grafico de bigotes para ver los precios más repetidos de los vinos y sus valores ouliers***

⬇️ ***Busco valores maximos y minimos de la columna "price"***

In [ ]:
max_price = df_final_limpio['price'].max()
min_price = df_final_limpio['price'].min()

print(f"Valor máximo de 'price': {max_price}")
print(f"Valor mínimo de 'price': {min_price}")


###2.1 A partir de generar recortes y etiquetas, genero un df en el que segmenta la columna "price" en rangos

In [ ]:
bins = [0, 50, 100, 200, 500, 1000, 2000, float('inf')]
labels = ['0-50', '51-100', '101-200', '201-500', '501-1000', '1001-2000', '2000+']

df_final_limpio['rango_de_precios'] = pd.cut(df_final_limpio['price'], bins=bins, labels=labels)

df_final_limpio[['price', 'rango_de_precios']].head()


###2.2 A partir de estos rangos ya generados, le asigno una etiqueta correspondiente

In [ ]:
df_final_limpio['rango_de_precios'] = pd.qcut(df_final_limpio['price'], q=4, labels=['Bajo', 'Medio-Bajo', 'Medio-Alto', 'Alto'])

df_final_limpio[['price', 'rango_de_precios']].head()


###Observo cuántos productos están en cada rango de precio


In [ ]:
rango_de_precios_organizados = df_final_limpio['rango_de_precios'].value_counts()
rango_de_precios_organizados.head()



***⬇️ Genero un grafico de barras para poder graficar los rangos creados y cantidad de vinos que los componen***

In [ ]:
plt.figure(figsize=(8,6))
sns.barplot(x=rango_de_precios_organizados.index, y=rango_de_precios_organizados.values, palette='viridis')

plt.title('Distribución de vinos por rango de recio')
plt.xlabel('Rango de precios')
plt.ylabel('Cantidad de vinos')
plt.xticks(rotation=45)
plt.show()


***¿Existe relación entre precios y puntaje?***

In [ ]:
df_relación = df_final_limpio.select_dtypes(include="number")

corr_matrix = df_relación.corr()

plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)

plt.title("Mapa de calor de correlaciones")
plt.show()


##2.3 Aplición técnicas de regresión para predecir el precio de un vino según sus características.

🧮 El modelo que se empleará es de treeforest debido a que la relación entre variables es no lineal

In [ ]:
X = df_final_limpio[["points"]]
y = df_final_limpio["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

y_pred_rf = rf_model.predict(X_test_scaled)

mae_rf = mean_absolute_error(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f"MAE: {mae_rf:.2f}")
print(f"MSE: {mse_rf:.2f}")
print(f"RMSE: {rmse_rf:.2f}")
print(f"R² Score: {r2_rf:.2f}")

plt.figure(figsize=(7,5))
sns.scatterplot(x=X_test["points"], y=y_test, label="Real", color="blue")
sns.scatterplot(x=X_test["points"], y=y_pred_rf, label="Predicción", color="red")
plt.xlabel("Points")
plt.ylabel("Price")
plt.title("Random Forest: Precio vs Puntos (Escalado)")
plt.legend()
plt.show()


🤔 Observo que el modelo tiene un mal rendimiento. Se equivoca en 15 unidades, penaliza los errores grandes y tiene errores significativos. A su vez, solo explica el 31% de los datos

💡 Opto por la idea de eliminar los valores outliers de la columna "price", mediante el IRQ, y generar el modelo de nuevo

In [ ]:
Q1_price = df_final_limpio["price"].quantile(0.25)
Q3_price = df_final_limpio["price"].quantile(0.75)

IQR_price = Q3_price - Q1_price

lower_bound_price = Q1_price - 1.5 * IQR_price
upper_bound_price = Q3_price + 1.5 * IQR_price

df_no_outliers_price = df_final_limpio[(df_final_limpio["price"] >= lower_bound_price) & (df_final_limpio["price"] <= upper_bound_price)]

print(f"Total de outliers eliminados en 'price': {len(df_final_limpio) - len(df_no_outliers_price)}")


In [ ]:
X_no_outliers = df_no_outliers[["points"]]
y_no_outliers = df_no_outliers["price"]
X_train_no_outliers, X_test_no_outliers, y_train_no_outliers, y_test_no_outliers = train_test_split( X_no_outliers, y_no_outliers, test_size=0.1, random_state=42)
scaler_no_outliers = StandardScaler()
X_train_scaled_no_outliers = scaler_no_outliers.fit_transform(X_train_no_outliers)
X_test_scaled_no_outliers = scaler_no_outliers.transform(X_test_no_outliers)
rf_model_no_outliers = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model_no_outliers.fit(X_train_scaled_no_outliers, y_train_no_outliers)

y_pred_rf_no_outliers = rf_model_no_outliers.predict(X_test_scaled_no_outliers)

mae_rf_no_outliers = mean_absolute_error(y_test_no_outliers, y_pred_rf_no_outliers)
mse_rf_no_outliers = mean_squared_error(y_test_no_outliers, y_pred_rf_no_outliers)
rmse_rf_no_outliers = np.sqrt(mse_rf_no_outliers)
r2_rf_no_outliers = r2_score(y_test_no_outliers, y_pred_rf_no_outliers)

print(f"MAE (sin outliers): {mae_rf_no_outliers:.2f}")
print(f"MSE (sin outliers): {mse_rf_no_outliers:.2f}")
print(f"RMSE (sin outliers): {rmse_rf_no_outliers:.2f}")
print(f"R² Score (sin outliers): {r2_rf_no_outliers:.2f}")

plt.figure(figsize=(7,5))
sns.scatterplot(x=X_test_no_outliers["points"], y=y_test_no_outliers, label="Real", color="blue")
sns.scatterplot(x=X_test_no_outliers["points"], y=y_pred_rf_no_outliers, label="Predicción", color="red")
plt.xlabel("Points")
plt.ylabel("Price")
plt.title("Random Forest: Precio vs Puntos (Sin Outliers)")
plt.legend()
plt.show()


😊 El modelo mejoró el rendimiento. Se equivoca en 10.19 unidades, reduce significativamente los errores y acierta de mejor manera las predicciones.

# 3. Análisis de Sentimiento en Reseñas (NLP)

In [ ]:
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

df_sample = df1.sample(5000, random_state=None).reset_index(drop=True)

texts = df_sample["description"].dropna().astype(str).tolist()

docs = list(nlp.pipe(texts, batch_size=50, n_process=4))

def limpiar_texto(doc):
    return " ".join([token.lemma_ for token in doc if not token.is_stop and token.is_alpha])

df_sample["description_clean"] = [limpiar_texto(doc) for doc in docs]

print(df_sample[["description", "description_clean"]].sample(5))


In [ ]:
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

df_sample = df1.sample(5000, random_state=42)

texts = df_sample["description"].astype(str).tolist()

docs = list(nlp.pipe(texts, batch_size=50, n_process=4))

def limpiar_texto(doc):
    return " ".join([token.lemma_ for token in doc if not token.is_stop and token.is_alpha])

df_sample["description_clean"] = [limpiar_texto(doc) for doc in docs]


df_bien = df_sample[df_sample["points"] >= 90]
df_mal = df_sample[df_sample["points"] < 83]

def contar_palabras(textos):
    palabras = " ".join(textos).split()
    return Counter(palabras).most_common(10)
top_bien = contar_palabras(df_bien["description_clean"])
top_mal = contar_palabras(df_mal["description_clean"])



In [ ]:

df_bien = df_sample[df_sample["points"] >= 90]
df_mal = df_sample[df_sample["points"] < 83]

def contar_palabras(textos):
    palabras = " ".join(textos).split()
    return Counter(palabras).most_common(10)

top_bien = contar_palabras(df_bien["description_clean"])
top_mal = contar_palabras(df_mal["description_clean"])

print("\n👍 Palabras más frecuentes en vinos bien puntuados (90+):")
print(top_bien)

print("\n👎 Palabras más frecuentes en vinos mal puntuados (<83):")
print(top_mal)
def generar_nube(textos, titulo):
    wordcloud = WordCloud(width=800, height=400, background_color="white").generate(" ".join(textos))

    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title(titulo, fontsize=14)
    plt.show()

generar_nube(df_bien["description_clean"], "Nube de Palabras - Vinos bien puntuados")

generar_nube(df_mal["description_clean"], " Nube de Palabras - Vinos mal puntuados")


## 3.1 Analisis de sentimientos de las reseñas

In [ ]:


df_sample["sentiment"] = np.where(df_sample["points"] >= 90, 1, 0)

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df_sample["description_clean"])

y = df_sample["sentiment"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"📊 Accuracy del modelo: {accuracy:.4f}")
print("\n🔍 Reporte de clasificación:")
print(classification_report(y_test, y_pred))


# 4. Sistema de Recomendación de Vinos


In [ ]:
le = LabelEncoder()
df1['variety_encoded'] = le.fit_transform(df1['variety'])
df1['country_encoded'] = le.fit_transform(df1['country'])
df1['province_encoded'] = le.fit_transform(df1['province'])



## 4.1 Sistema de recomendación por similitud de caracteristicas

In [ ]:
content_features = ['variety', 'country', 'province', 'region_1']

df1['content'] = df1[content_features].fillna('').agg(' '.join, axis=1)

vectorizer = TfidfVectorizer(stop_words='english')
X_content = vectorizer.fit_transform(df1['content'])

knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)

knn.fit(X_content)

def recomendar_vinos_knn(vino_index, k=5):
    distances, indices = knn.kneighbors(X_content[vino_index], n_neighbors=k+1)
    return indices[0][1:], distances[0][1:]

vino_index = int(input("Ingrese el índice del vino que desea analizar: "))

indices, distancias = recomendar_vinos_knn(vino_index, 5)

print(f"\n🔹 Recomendaciones para el vino {vino_index}:")
for i, (indice, distancia) in enumerate(zip(indices, distancias), 1):
    print(f"{i}. Vino {indice} (Similitud: {1 - distancia:.4f})")


## 4.1 Sistema de recomendación por similitud de caracteristicas entre catador y uva

In [ ]:
def recomendar_mixto(user, wine_title, k=5):
    if user in df1["taster_name"].unique():
        rec_colab = recomendar_vino_svd(user, k)
    else:
        rec_colab = []

    if wine_title in df1["title"].unique():
        rec_content = recomendar_vinos_knn(df1[df1["title"] == wine_title].index[0], k)
    else:
        rec_content = []

    recomendaciones = list(set(rec_colab + list(rec_content)))
    return recomendaciones[:k]

print(recomendar_mixto("Robert Parker", "Château Margaux", 5))


In [ ]:
df1_ratings_unicos = df1.drop_duplicates(subset=["taster_name", "title"])
ratings_matrix = df1_ratings_unicos.pivot(index='taster_name', columns='title', values='points')

user_similarity = cosine_similarity(ratings_matrix.fillna(0))
user_similarity_df = pd.DataFrame(user_similarity, index=ratings_matrix.index, columns=ratings_matrix.index)
user_similarity_df.head(40)

*En este caso, se puede cambiar nombres de catadores para buscar aquellos que, en caso de tener gusto similar al quien se nombra, el sistema recomendará vinos cuyos gustos sean similares* 😎

In [ ]:
def recomendar_vinos(usuario, n_recomendaciones=5):
    similar_users = user_similarity_df[usuario].sort_values(ascending=False)
    similar_users = similar_users.iloc[1:]
    recommended_wines = []
    for user in similar_users.index:
        user_ratings = ratings_matrix.loc[user]
        unscored_wines = user_ratings[user_ratings.isna()].index
        recommended_wines.extend(unscored_wines)
    recommended_wines_count = pd.Series(recommended_wines).value_counts().head(n_recomendaciones)
    return recommended_wines_count
recomendaciones = recomendar_vinos("Virginie Boone", 10)
print(recomendaciones)
